# Week 12 - Moar Pandas


- Subsetting data
    - Masking by values
- Merging Data
    - Merging and Joining
    - Joining with Real Data
- Grouping data
    - Split Apply Combine
    - Split Apply Combine with Real Data

In [ ]:
# Import pandas so we can do stuff
import pandas as pd


## Subsetting Data

* It is sometimes helpful to think of a Pandas Dataframe as a little database. 
* There is data and information stored in the Pandas Dataframe (or Series) and you want to *retrieve* it.
* Pandas has multiple mechanisms for getting specific bits of data and information from its data structures. 

### Masking: Filtering by Values

* The most common is to use *masking* to select just the rows you want. 
* Masking is a two stage process, first you create a sequence of boolean values (corresponding to rows in your data) based upon a conditional expression--which you can think of as a "query"--and then you index your dataframe using that boolean sequence. 

In [ ]:
# read the data into a pandas dataframe
order_data  = pd.read_csv("files/chipotle.tsv", sep="\t")
# inspect the dataframe
order_data.head() 

In [ ]:
# Let's look at the chipotle order data
order_data.head(10)

In [ ]:
# Let's look at all the columns
order_data.info()

* How might we only look at particular orders?
* First step is to create a *query mask*, a list of `True/False` values for rows that satisfy a particular condition.

In [ ]:
# create a query mask for chicken bowls
query_mask = order_data['item_name'] == "Chicken Bowl"

#look at the first 20 items to see what matches
query_mask.head(20)

* This tells us the row id and True or False if the item type equals chicken bowl
* We can look up that row by index and see if it is correct

In [ ]:
order_data.iloc[19]

* Yup! So now that we know the mask works, we can create a *subset* of our data containing chicken bowls.

In [ ]:
chicken_bowls = order_data[query_mask]
chicken_bowls.head()

* Now you can do things like calculate the average price for chicken bowl orders

In [ ]:
# Calculate the mean price for chicken bowls
chicken_bowls['item_price'].mean()

In [ ]:
# See how many chicken bowls people order
chicken_bowls['quantity'].value_counts()

* We can also combine query masks using boolean logic
* Can we look at just the chicken bowl orders that were less than $10

In [ ]:
# create a query mask for chicken bowls
item_query_mask = order_data['item_name'] == "Chicken Bowl"
# create a query mask for cheap orders
price_query_mask = order_data['item_price'] < 10

# apply both query masks using boolean AND
cheap_chicken_bowls = order_data[item_query_mask & price_query_mask]
cheap_chicken_bowls.head()

In [ ]:
# Median price for cheap chicken bowls
cheap_chicken_bowls['item_price'].median()

* Query masks can be used to filter and create subsets of data
* Note, this method of subsetting data creates what is called a "view" of the data
* You are basically working with a big slice of the original dataframe, not a separate copy of the data
* This means if you try an do transformations on that view, you will get an error
* For more information, [see the pandas documentation](http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy)

In [ ]:
cheap_chicken_bowls['half_price'] = cheap_chicken_bowls['item_price'] / 2

In [ ]:
copy_of_cheap_chicken_bowls = cheap_chicken_bowls.copy()
copy_of_cheap_chicken_bowls['half_price'] = copy_of_cheap_chicken_bowls['item_price'] / 2
copy_of_cheap_chicken_bowls.head()

## Merging Data

* Bringing disparate datasets together is one of the more powerful features of Pandas
* Like with Python lists, you can `append()` and `concat()` Pandas `Series` and `Dataframes`
* The `concat` is a module function, you call it directly from the pandas module (usually called `pd`)

In [ ]:
# concatinate two series together
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2]) #note the Seres are passed as a list

In [ ]:
# order matters
pd.concat([ser2, ser1])

In [ ]:
# concatinate two dataframes
df1 = pd.DataFrame({"A":["A1", "A2"],
                    "B":["B1","B2"]},index=[1,2])
df2 = pd.DataFrame({"A":["A3", "A4"],
                    "B":["B3","B4"]},index=[3,4])
pd.concat([df1,df2])

* Pandas will automatically line up matching indexes

In [ ]:
# concatinate dataframes horizontally
df1 = pd.DataFrame({"A":["A1", "A2"],
                    "B":["B1","B2"]},index=[1,2])
df2 = pd.DataFrame({"C":["C1", "C2"],
                    "D":["D1","D2"]},index=[1,2])
pd.concat([df1,df2], axis=1)

* And pandas will gracefully handle mis-alignment

In [ ]:
# What happens when indexes don't line up
df1 = pd.DataFrame({"A":["A1", "A2"],
                    "B":["B1","B2"]},index=[1,2])
df2 = pd.DataFrame({"A":["A3", "A4"],
                    "B":["B3","B4"]},index=[3,4])
pd.concat([df1,df2], axis=1)

* The `append` function is a method of a Series/Dataframe and returns a new object

In [ ]:
# append df2 to df1
df1.append(df2)

### Merging and Joining

* While `concat()` is useful it lacks the power to do complex data merging
* For example, I have two tables of different data but one shared column
* This is where the `merge()` function becomes useful because it lets you *join* datasets
* The concept of "join" has lots of theory and is a richly developed method for *joining* data

#### One-to-one joins

In [ ]:
# create two dataframes with one shared column
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue', "Nancy"],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR', "Librarian"]})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})

In [ ]:
# display df1
df1

In [ ]:
# display df2
df2

In [ ]:
# merge df1 and df2 into a new dataframe df3
df3 = pd.merge(df1, df2)
df3

* The new dataframe `df3` now has all of the data from df1 and df2
* The `merge` function automatically connected the two tables on the "employee" column
* But what happens when your data don't line up?

#### Many-to-one joins

* Sometimes there isn't a one to one relationshp between rows in  two datasets
* A *many-to-one* join lets you combine these datasets

In [ ]:
df3

In [ ]:
# make another dataframe about the supervisor for each group
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
df4

In [ ]:
# Merge df3 from above with the supervisor info in df4
pd.merge(df3,df4)

* Notice how the information about Guido, the manager for Engineering, is repeated.
* Pandas automatically fills in these values to maintain the tabular, 2 dimensional structure of the data
* While this might seem like duplicated data, it makes it easier to quickly look up Jake and Lisa's supervisor without consulting multiple tables

#### Many-to-many joins

* Let's combine the employee information with skills information
* Notice there isn't a one to one or even a one to many relationship between these tables
* Each group can have multiple skills, so **what do you think will happen?**

In [ ]:
# Use the employee table specified above
df1

In [ ]:
# create a new dataframe with skills information
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR', 'Librarian'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization', 'nunchucks']})
df5

In [ ]:
pd.merge(df1, df5)

* Amazing, Pandas merge capabilities are very useful when column names match up
* But what do you do if the names of your columns don't match?
* You could change column names...
* But that is crazy! Just use the `left_on` and `right_on` parameters to the `merge()` function

In [ ]:
# Use the employee table specified above
df1

In [ ]:
# rename the column "employee" to "name"
df2 = df2.rename({"employee":"name"}, axis="columns")
df2

In [ ]:
# lets try and merge them without specifying what to merge on
pd.merge(df1, df2)

* Gak, error! Pandas can't figure out how to combine them
* What are the column names I should specify?

In [ ]:
# Now lets specify the column name 
pd.merge(df1, df2, left_on="employee", right_on="name" )

* Notice we now have a redundant employee/name column, this is a by-product of merging different columns
* If you want to get rid of it you can use the `drop` method

In [ ]:
# drop the name column, axis=1 means axis='col', which is confusing
pd.merge(df1, df2, left_on="employee", right_on="name" ).drop("name", axis=1)

### Joining with Real Data

The example above was illustrative, but it might be useful to see how you might join two *real* datasets together 

We have two data files, `service_requests.csv`, which contains 311 requests and `311-codebook.csv`, which maps request types/issues to higher level categories. 

In [ ]:
# load the 311 data
service_requests = pd.read_csv("files/service_requests.csv")
service_requests.head()

In [ ]:
# look at all the unique values for REQUEST TYPE
service_requests["REQUEST_TYPE"].unique()

In [ ]:
# how many unique request types are there?
len(service_requests["REQUEST_TYPE"].unique())

If we are aggregating our data, 294 separate issues is still a lot of data!

In [ ]:
# load the 311 codebook data
code_book = pd.read_csv("files/311-codebook.csv")
code_book.head()

So now we can do some fancy merging magic. We know there is some overlap in the data column between the 311 requests and the codebook. Basically, the data in the `REQUEST_TYPE` column of the 311 data shares the same values as the `Issue` column of the code book.

In [ ]:
# create a new, merged dataframe t
merged_df = pd.merge(service_requests, code_book, left_on="REQUEST_TYPE", right_on="Issue")
merged_df.head()

In [ ]:
# What are all the unique values for Category
merged_df["Category"].unique()

In [ ]:
# How many categories
len(merged_df["Category"].value_counts())

In [ ]:
# Count the number of service requests for each category
merged_df["Category"].value_counts()

This is a much more comprehendable aggregation

## Grouping Data


* A common pattern in data analysis is splitting data by a key and then performing some math on all of the values with that key and finally combining it all back together
* This is commonly known in data circles as *split, apply, combine*


In [ ]:
# create a dataframe to illustrate GroupBy
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6),
                   'counts':[45,234,6,2,1324,345], 
                   'things':['dog', 'cat', 'cat', 'dog', 'cat', 'cat']}
                 )
df

* Under the hood Pandas is creating a bunch of new Dataframes based on the grouping column values

In [ ]:
# Loop over each group of data, don't do this at home
for group in df.groupby('key'):
    print("Group for key:", group[0])
    print("Data:", group[1])
    print("Data Type:", type(group[1]))
    print()

* Cool, we can see that we have *split* our data into three groups
* Now, we need to tell Pandas what function to *apply* to each group
* We need to specify what kind of aggregation, transformation, or computation to perform on the group

In [ ]:
# Tell pandas to add up all of the values for each key
df.groupby('key').sum()

* Pandas will apply the aggregation function only to relevant columns
* Mathy functions will only be applied to numerical columns

In [ ]:
# you can save the group object and run different aggregations
grouped_dataframe = df.groupby('key')
grouped_dataframe.sum()

In [ ]:
grouped_dataframe.mean()

In [ ]:
grouped_dataframe.prod()

* The following table summarizes some other built-in Pandas aggregations:

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``size()``               | Total number of items w/ NaNs   |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |

* These are all methods of ``DataFrame`` and ``Series`` objects.

* You can also do multiple levels of grouping

In [ ]:
df.groupby(['things','key']).count()

* What you are seeing is what is called a [Multilevel Index](https://pandas.pydata.org/pandas-docs/stable/advanced.html)
* Sadly, we don't have time to cover that topic, but this chapter on [Hierarchical Indexing](https://jakevdp.github.io/PythonDataScienceHandbook/03.05-hierarchical-indexing.html) in the [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/) is a great introduction to the topic.

## Split, Apply, Combine with Real Data

Let's try and analyze some real data about the all those staircases in Pittsburgh.

In [ ]:
# load the PGH steps data
steps = pd.read_csv("files/steps.csv")
steps.head()

With the data loaded into pandas we can start asking questions of our data

In [ ]:
# How many steps are there total in PGH?
steps["number_of_steps"].sum()

In [ ]:
# Count the number of staircases with each material type
steps['material'].value_counts()

What if we want to break this down by neighborhood?

In [ ]:
# group by neighborhood, then count the material types and display 50 values
steps.groupby("neighborhood")['material'].value_counts().head(50)

You could also use indexing to grab values for a particular neighborhood

In [ ]:
# group by neighborhood, then count the material types and display 50 values
steps.groupby("neighborhood")['material'].value_counts().loc['Greenfield']

In [ ]:
# What is the average number of steps
steps["number_of_steps"].mean()

In [ ]:
# What is the average number of steps by material
steps.groupby("material")['number_of_steps'].mean()

In [ ]:
# What neighborhood has the most number of steps
steps.groupby("neighborhood")["number_of_steps"].sum().sort_values(ascending=False)

In [ ]:
# What is the longest staircase by type per neighborhood
steps.groupby(["neighborhood", "material"])["number_of_steps"].max().head(50)